# MPh API 全覆盖测试（基于 tritum_cycle_Model.mph）

本笔记本专注于对 MPh 核心 API 进行系统性测试。
执行前请确保 COMSOL 可用且许可有效。

In [ ]:
import mph
import os
from pprint import pprint

# 启动 Comsol 客户端
try:
    client = mph.start()
    print(f"MPh {mph.__version__} connected to Comsol {client.version}")
except Exception as e:
    raise RuntimeError(f"Connection failed: {e}")

# 自动查找并加载模型
model_name = 'tritum_cycle_Model.mph'
search_paths = [
    model_name,
    os.path.join('..', 'experiment_00', model_name),
    r'D:\\Administrator\\Documents\\Github\\tricys_mph\\experiment_00\\tritum_cycle_Model.mph',
]

model = None
for p in search_paths:
    if os.path.exists(p):
        print(f"Loading model from: {p} ...")
        model = client.load(p)
        break

if not model:
    raise FileNotFoundError(f"无法找到 {model_name}，请检查路径。")

print("Model loaded successfully.")

## 1. MPh API 全覆盖测试（基于 tritum_cycle_Model.mph）

以下小节覆盖配置、客户端、模型、节点、参数、评估与清理等核心 API。

In [ ]:
# 1.1 配置 API：mph.config.option
print("Current options:")
pprint(mph.config.option(None, None))

# 示例：临时启用缓存（可按需关闭）
mph.config.option('caching', True)
print("Caching enabled:", mph.config.option('caching', None))

In [ ]:
# 1.2 Client API：会话信息、已加载模型、许可模块
print(client)
print("Standalone:", client.standalone)
print("Version:", client.version)
print("Loaded models:", client.names())
print("Licensed modules (subset):", client.modules()[:10])

In [ ]:
# 1.3 Model API：基础信息与问题检测
print("Model name:", model.name())
print("Model file:", model.file())
print("Model version:", model.version())
print("Model modules:", model.modules())

problems = model.problems()
print("Problems:", "None" if not problems else f"{len(problems)} issue(s)")
if problems:
    print(problems[0])

In [ ]:
# 1.4 参数读写 API（带恢复）
original_tbr = model.parameter('TBR')
print("Original TBR:", original_tbr)

model.parameter('TBR', '1.15')
print("Updated TBR:", model.parameter('TBR'))

# 恢复原值
model.parameter('TBR', original_tbr)
print("Restored TBR:", model.parameter('TBR'))

# 批量读取参数（不评估与评估）
params_raw = model.parameters(evaluate=False)
params_eval = model.parameters(evaluate=True)
print("Param count:", len(params_raw))
print("TBR raw/eval:", params_raw.get('TBR'), params_eval.get('TBR'))

In [ ]:
# 1.5 Node API：路径导航与属性读写
root = model/None
functions = model/'functions'
physics = model/'physics'
studies = model/'studies'

print("Root exists:", root.exists())
print("Functions children:", [c.name() for c in functions][:5])
print("Physics children:", [c.name() for c in physics][:5])
print("Studies:", [c.name() for c in studies])

# 读取一个研究时间节点的属性（tlist）
time_node = None
for study in studies:
    for feat in study:
        try:
            if feat.java.tag() == 'time':
                time_node = feat
                break
        except Exception:
            pass
    if time_node:
        break

if time_node:
    print("Time node:", time_node)
    print("tlist:", time_node.property('tlist'))
else:
    print("Time node not found.")

In [ ]:
# 1.6 Node.create / remove：临时创建并清理功能节点
tmp_name = 'api_temp_func'
tmp = model/'functions'/tmp_name

if tmp.exists():
    tmp.remove()

try:
    tmp = (model/'functions').create('Analytic', name=tmp_name)
    tmp.property('funcname', 'f_api_test')
    tmp.property('expr', '1')
    tmp.property('fununit', '1')
    print("Created:", tmp, "tag=", tmp.tag())
finally:
    if (model/'functions'/tmp_name).exists():
        (model/'functions'/tmp_name).remove()
        print("Removed temp function node.")

In [ ]:
# 1.7 Model.evaluate：结果提取（若已有解）
try:
    # t 为内置时间变量，若解已存在可直接评估
    time_h = model.evaluate('t', unit='h')
    i15 = model.evaluate('I15', unit='g')
    print("Eval OK:", len(time_h), len(i15))
except Exception as e:
    print("Evaluation skipped:", e)

In [ ]:
# 1.8 清理测试：仅清除结果缓存（不修改模型结构）
try:
    model.clear()
    print("Cleared stored data (plots/solutions/meshes).")
except Exception as e:
    print("Clear skipped:", e)

## 2. 进阶补充：树检视、导出与保存

以下内容补充 `tree()` / `inspect()`、导出节点与保存 API 的基本用法。

In [ ]:
# 2.1 tree() / inspect()：模型树与节点自省
import mph

# 打印模型树（深度可调）
try:
    mph.tree(model, max_depth=3)
except Exception as e:
    print("tree() skipped:", e)

# 选取一个节点进行 inspect（示例：studies 分组）
try:
    studies = model/'studies'
    mph.inspect(studies)
except Exception as e:
    print("inspect() skipped:", e)

In [ ]:
# 2.2 Client.create / remove：创建并清理临时模型
try:
    temp_model = client.create("api_temp_model")
    print("Temp model created:", temp_model.name())
finally:
    try:
        client.remove("api_temp_model")
        print("Temp model removed.")
    except Exception as e:
        print("Temp model cleanup skipped:", e)

In [ ]:
# 2.3 Model.export：导出结果（若存在导出节点）
import os

exports = model.exports()
print("Export nodes:", exports)

if exports:
    export_name = exports[0]
    out_path = os.path.join(os.getcwd(), "api_export_test.png")
    try:
        model.export(export_name, file=out_path)
        print("Exported to:", out_path)
    except Exception as e:
        print("Export failed:", e)
else:
    print("No export nodes found; export skipped.")

In [ ]:
# 2.4 Model.save：另存为（避免覆盖原文件）
import os

save_path = os.path.join(os.getcwd(), "api_save_test.mph")
try:
    model.save(save_path)
    print("Saved model to:", save_path)
except Exception as e:
    print("Save skipped:", e)

## 3. API 总结归纳（速查）

下面按功能域归纳本笔记本涉及的 MPh 主要 API：

### 会话与配置
- `mph.start()`：启动本地会话并返回 `Client`。
- `mph.config.option()`：读取/设置配置项（如 `session`、`caching`、`classkit`）。

### Client（客户端）
- `Client.names()` / `Client.models()`：列出已加载模型。
- `Client.load(path)`：加载 `.mph` 模型文件。
- `Client.create(name)`：创建空模型。
- `Client.remove(model|name)` / `Client.clear()`：移除模型或清空内存模型。
- `Client.modules()` / `Client.cores` / `Client.version`：查询许可模块与会话信息。
- `Client.connect()` / `Client.disconnect()`：连接/断开 Comsol Server（如使用 client–server 模式）。

### Model（模型）
- 结构与信息：`name()`、`file()`、`version()`、`modules()`、`problems()`。
- 求解流程：`build()`、`mesh()`、`solve()`。
- 参数读写：`parameter()`、`parameters()`、`description()`。
- 节点代理：`property()`、`properties()`、`create()`、`remove()`。
- 结果评估：`evaluate()`（支持 `dataset`、`inner`、`outer`）。
- 导入导出：`import_()`、`export()`、`save()`。
- 维护操作：`clear()`、`reset()`。

### Node（节点）
- 导航：`parent()`、`children()`、`exists()`、`name()`、`tag()`、`type()`。
- 属性：`property()`、`properties()`、`comment()`。
- 选择集：`select()`、`selection()`。
- 运行与开关：`run()`、`toggle()`。
- 结构变更：`create()`、`remove()`、`rename()`、`retag()`。

### 工具函数
- `mph.tree(node, max_depth=...)`：打印模型树结构。
- `mph.inspect(node|java)`：打印 Java 节点对象的方法/属性摘要。

> 提示：推荐使用 `tag` 定位节点；在脚本中比 `name` 更稳定。